<a href="https://colab.research.google.com/github/astorguy/learn_ngspice/blob/main/notebooks/rlc/rlc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set-up
Installs and imports.

This notebook can be run in Google Colab or in Jupyter. However, the setup is slightly different. This function determines which environment we are using.

In [1]:
def is_running_in_colab() -> bool:
    try:
        import google.colab  # type: ignore
        return True
    except ImportError:
        return False

In [5]:
import subprocess, sys

if not is_running_in_colab():
    subprocess.run(["sudo", "apt-get", "update"], check=True)
    subprocess.run(["sudo", "apt-get", "install", "-y", "ngspice"], check=True)
else:
    subprocess.run(["apt-get", "install", "-y", "ngspice"], check=True)

subprocess.check_call([sys.executable, "-m", "pip", "install", "py4spice"])

Get:1 http://deb.debian.org/debian trixie InRelease [140 kB]
Get:2 http://deb.debian.org/debian trixie-updates InRelease [47.3 kB]
Get:3 http://deb.debian.org/debian-security trixie-security InRelease [43.4 kB]
Get:4 http://deb.debian.org/debian trixie/main amd64 Packages [9670 kB]
Get:5 http://deb.debian.org/debian trixie-updates/main amd64 Packages [5412 B]
Get:6 http://deb.debian.org/debian-security trixie-security/main amd64 Packages [100 kB]
Fetched 10.0 MB in 1s (10.6 MB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libxaw7 libxmu6 libxpm4
Suggested packages:
  ngspice-doc
The following NEW packages will be installed:
  libxaw7 libxmu6 libxpm4 ngspice
0 upgraded, 4 newly installed, 0 to remove and 0 not upgraded.
Need to get 4554 kB of archives.
After this operation, 30.5 MB of additional disk space will be used.
Get:1 http://deb.debian.org/debian trixie/main amd

dpkg-preconfigure: unable to re-open stdin: No such file or directory


Fetched 4554 kB in 0s (71.0 MB/s)
Selecting previously unselected package libxmu6:amd64.
(Reading database ... 34693 files and directories currently installed.)
Preparing to unpack .../libxmu6_2%3a1.1.3-3+b4_amd64.deb ...
Unpacking libxmu6:amd64 (2:1.1.3-3+b4) ...
Selecting previously unselected package libxpm4:amd64.
Preparing to unpack .../libxpm4_1%3a3.5.17-1+b3_amd64.deb ...
Unpacking libxpm4:amd64 (1:3.5.17-1+b3) ...
Selecting previously unselected package libxaw7:amd64.
Preparing to unpack .../libxaw7_2%3a1.0.16-1_amd64.deb ...
Unpacking libxaw7:amd64 (2:1.0.16-1) ...
Selecting previously unselected package ngspice.
Preparing to unpack .../ngspice_44.2+ds-1_amd64.deb ...
Unpacking ngspice (44.2+ds-1) ...
Setting up libxmu6:amd64 (2:1.1.3-3+b4) ...
Setting up libxpm4:amd64 (1:3.5.17-1+b3) ...
Setting up libxaw7:amd64 (2:1.0.16-1) ...
Setting up ngspice (44.2+ds-1) ...
Processing triggers for man-db (2.13.1-1) ...
Processing triggers for libc-bin (2.41-12+deb13u1) ...
Defaulting to


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


0

In [6]:
from pathlib import Path
from types import SimpleNamespace
import py4spice

# Ngspice simulation of RLC

### Define paths
There are paths and files to define. We're put them into a namespace to keep things organized. Two subdirectories are also defined:
- netlists
- results

In [7]:
paths = SimpleNamespace() # create the namespace for the class

paths.ngspice = Path("/usr/bin/ngspice")
paths.workspace = Path.cwd()
paths.netlists = paths.workspace /Path("netlist")
paths.results = paths.workspace / Path("results")
paths.top = paths.netlists / Path("top.cir")
paths.sim_transcript = paths.results / Path("sim_transcript.log")

# create these directories if the don't yet exist
paths.netlists.mkdir(parents=True, exist_ok=True)
paths.results.mkdir(parents=True, exist_ok=True)

# create an empty file for sim_transcript
paths.sim_transcript.write_text("")

0

### Netlists namespace
Netlist objects are created. These netlist snippets will be used later to create a single top-level netlist which will be read in during the batch simulation. Again, we put them into a namespace to keep things organized.

In [8]:
netlists = SimpleNamespace() # create the namespace for the class

netlists.blankline = py4spice.Netlist("")
netlists.title = py4spice.Netlist("* my ngspice netlist")
netlists.end = py4spice.Netlist(".end")

### Creating DUT (Device Under Test)
Though this circuit is simple, we are going to separate the DUT from all the other parts of the netlist. The simulations, title, and `.end` lines will be added separately.

We want to measure the current through the capacitor and the inductor. Therefore, we do the old spice trick of adding 0 volt source in series. (Ugly, but effective)

In [9]:
dut_contents = """
r1 out 0 100
l1 out lmid 10e-6 ic=300e-3
vil lmid 0 dc 0
c1 out cmid 10e-9
vic cmid 0 dc 0
""".strip()

netlists.dut = py4spice.Netlist(dut_contents)

### Vectors dictionary
Vectors objects are used to define a set of circuit signals to create and display. Again, we put them into a dictionary to keep things organized.

In [15]:
vectors = SimpleNamespace() # create the namespace for the class

vectors.all = py4spice.Vectors("all")
vectors.out = py4spice.Vectors("out")
vectors.vil_branch = py4spice.Vectors("vil#branch")
vectors.vic_branch = py4spice.Vectors("vic#branch")
vectors.currents = vectors.vil_branch + vectors.vic_branch

vectors.I_capacitor = py4spice.Vectors("I_capacitor") # rename for nice plots
vectors.I_inductor = py4spice.Vectors("I_inductor") # rename for nice plots
vectors.currents_renamed = vectors.I_capacitor + vectors.I_inductor

### Analyses to run during the simulation process
Each batch simulation in Ngspice can perform one or more analyses. We will create a list of analyses to make it easier to reuse the code. The Analysis object has the following arguments:

| Argument | Description |
| :--- | :--- |
| **name** | Label given to the object. |
| **cmd_type** | Ngspice has different types of analyses. The `op`, `tran`, `ac`, and `dc` analyses are supported with `py4spice`. |
| **cmd** | This is the actual command that will be sent to Ngspice. Its arguments can be found in the Ngspice documentation. |
| **vector** | This defines which signals will be in the simulation results. |
| **results_loc** | Where to put the results files. |

In [16]:
list_of_analyses = []  # start with an empty list

# 1st analysis: operating point
tran1 = py4spice.Analyses(
    name="tran1",
    cmd_type="tran",
    cmd="tran 1e-9 5e-6 uic",
    vector=vectors.all,
    results_loc=paths.results,
)
list_of_analyses.append(tran1)


### Control section
This is the section that tells Ngspice what operations to do. A netlist object snippet is created. It will be concatenated with the other netlist objects to create the top-level netlist.

In [ ]:
my_control = py4spice.Control()  # create 'my_control' object

# add all the analyses defined above into the control section
for analysis in list_of_analyses:
    my_control.insert_lines(analysis.lines_for_cntl())

# convert control section into a netlist object
netlists["control1"] = py4spice.Netlist(str(my_control))

### Create a single netlist, ready for simulation
The netlist snippets are concatenated and merged into `top`. `top.cir` is written out. It will be read in during the simulation. We print out `top` to verify everything is correct.

In [ ]:
# concatenate all tne netlists to make top1 and add to netlist dict
netlists["top"] = (
    netlists["title"]
    + netlists["blankline"]
    + netlists["dut"]
    + netlists["blankline"]
    + netlists["control1"]
    + netlists["blankline"]
    + netlists["end_line"]
    + netlists["blankline"]
    )

# write netlist to a file so ngspice can read it
paths["top"] = paths["netlists"] / "top.cir"
netlists["top"].write_to_file(paths["top"])

# here is the netlist in top.cir and will be used for simulation
print(netlists["top"])

### Simulate
The Ngspice command is constructed, printed out for verification, and executed. `timeout` (in seconds) is set in case the simulation hangs.

In [ ]:
# prepare simulate object and simulate
sim = py4spice.Simulate(
    ngspice_exe=paths["ngspice"],
    netlist_filename=paths["top"],
    transcript_filename=paths["sim_transcript"],
    name="sim1",
    timeout=5,
)
# print out the command that will be used in the spawned process
print(sim.ngspice_command)

sim.run()  # run the Ngspice simulation

### Create simResults objects
The raw results we get from Ngspice are large text files. We convert these to py4spice simResults objects so we can easily manipulate and display.

In [ ]:
sim_results: list[py4spice.SimResults] = [
    py4spice.SimResults.from_file(analysis.cmd_type, analysis.results_filename)
    for analysis in list_of_analyses
]

### Create Waveforms
Create waveforms from simulation data

In [ ]:
# prepare waveform for current through inductor
out_waves = py4spice.Waveforms(sim_results[0].header, sim_results[0].data_plot)
out_waves.vec_subset(vectors["currents"].list_out())
out_waves.scaler(1.0, "vil#branch", "I_inductor")  # rename
out_waves.scaler(1.0, "vic#branch", "I_capacitor")  # rename

### Plot
Right-click on plot and "Open image in new tab"

In [ ]:
plot_data = out_waves.x_axis_and_sigs(vectors["currents_renamed"].list_out())
y_names = vectors["currents_renamed"].list_out()
my_plt = py4spice.Plot("tr_plt", plot_data, y_names, paths["results"])
my_plt.set_title("RLC Transient Simulation Results")
my_plt.define_axes(("time", "sec", "linear"), ("amps", "A", "linear"))